### Paxillin movie segmentation and object feature analysis

with Annabel's 3D fixed image

Liya Ding, 2024.03

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import umap
from joblib import dump, load
import os.path

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.transforms import Bbox

import tifffile 
import czifile

import skimage
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage.filters import (threshold_otsu, threshold_niblack,
                             threshold_sauvola)

from skimage.morphology import binary_opening, binary_dilation
from skimage.morphology import disk


from scipy import ndimage
from scipy.ndimage import distance_transform_cdt
from scipy.ndimage import gaussian_filter

from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# function from aicssegmentation tool
from utils.vessel_2d import filament_2d_wrapper
from utils.pre_processing_utils import intensity_normalization


In [2]:
czimovie_dir =  '/mnt/d/lding/FA/data/FHL2-KI-U2OS-Actin405-Pax568-endoFHL2488_annabel_20240226'

csv_output_dir = '/mnt/d/lding/FA/analysis_results/pax_3D/pax_3D_0226/SM_Vess-LocThr-GlThr-Open_csv'
plot_output_dir = '/mnt/d/lding/FA/analysis_results/pax_3D/pax_3D_0226/SM_Vess-LocThr-GlThr-Open_plot'
seg_output_dir = '/mnt/d/lding/FA/analysis_results/pax_3D/pax_3D_0226/SM_Vess-LocThr-GlThr-Open_seg'

pixel_size = 0.0775
time_point = 0

if not os.path.isdir(csv_output_dir):
    os.makedirs(csv_output_dir)
if not os.path.isdir(plot_output_dir):
    os.makedirs(plot_output_dir)
if not os.path.isdir(seg_output_dir):
    os.makedirs(seg_output_dir)

In [4]:
### for each movie, make sub-folders for different output for easier viewing in imagej

movie_mip_output_dir = os.path.join(seg_output_dir,  'mip')
if not os.path.isdir(movie_mip_output_dir):   os.makedirs(movie_mip_output_dir)
   

filenames = [x for x in os.listdir(czimovie_dir) if os.path.isfile(os.path.join(czimovie_dir, x)) and ('.czi' in x)]

for filenameID in range(len(filenames)):
  
    filename = filenames[filenameID]
    img = czifile.imread(os.path.join(czimovie_dir,filename))
    pax_img = img[0,0,0,:,:,:,0].squeeze()
    
    MIP_pax_ori_img = pax_img.max(axis=0)
    tifffile.imsave(os.path.join(movie_mip_output_dir, 'SS_pax_img_'+filename+'_MIP'+'.tif'),MIP_pax_ori_img.astype(np.float32))   
  